# 1.DTM 예제

## 1.1 sklearn

In [1]:
# 필요한 라이브러리 불러오기
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

# 샘플 문서 리스트 (겹치는 단어가 포함된 데이터)
documents = [
    "오늘 날씨는 매우 좋다. 나는 기분이 좋다.",
    "오늘 커피를 마셨다. 커피는 정말 좋다.",
    "주말에 가족과 산책을 했다. 오늘은 날씨가 좋다.",
    "나는 책을 좋아한다. 독서는 매우 좋다.",
    "오늘 저녁에 영화를 볼 예정이다. 나는 영화를 좋아한다."
]
DTM = count_vect.fit_transform(documents)
DTM.toarray()

import pandas as pd

# 문서 이름 리스트 (행 인덱스)
doc_names = ['문서' + str(i+1) for i in range(len(documents))]
doc_names

# 단어 사전의 인덱스를 기준으로 정렬된 단어 리스트 생성
vocab = count_vect.get_feature_names_out()
vocab

# DTM 행렬을 데이터프레임으로 변환
dtm_df = pd.DataFrame(DTM.toarray(), index=doc_names, columns=vocab)


# DataFrame 출력
display(dtm_df)

,가족과,기분이,나는,날씨가,날씨는,독서는,마셨다,매우,산책을,영화를,...,오늘은,저녁에,정말,좋다,좋아한다,주말에,책을,커피는,커피를,했다
문서1,0,1,1,0,1,0,0,1,0,0,...,0,0,0,2,0,0,0,0,0,0
문서2,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,0,0,1,1,0
문서3,1,0,0,1,0,0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,1
문서4,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,1,0,1,0,0,0
문서5,0,0,1,0,0,0,0,0,0,2,...,0,1,0,0,1,0,0,0,0,0


## 1.2.gensim

In [9]:
# 필요한 라이브러리 불러오기
import gensim
from gensim import corpora
import numpy as np
import pandas as pd

# 샘플 문서 리스트 (토큰화된 형태가 아닌 1차원 리스트 형태)
documents = [
    "오늘 날씨는 매우 좋다. 나는 기분이 좋다.",
    "오늘 커피를 마셨다. 커피는 정말 좋다.",
    "주말에 가족과 산책을 했다. 오늘은 날씨가 좋다.",
    "나는 책을 좋아한다. 독서는 매우 좋다.",
    "오늘 저녁에 영화를 볼 예정이다. 나는 영화를 좋아한다."
]

# 1. 문서를 공백으로 토큰화
doc_ls = [doc.split() for doc in documents]


# 2. Gensim의 사전을 생성
id2word = corpora.Dictionary(doc_ls)

# 3. 각 문서를 BoW 형식으로 변환 (문서-단어 행렬)
DTM =[ id2word.doc2bow(doc) for doc in doc_ls]

# 4. BoW 형식을 풀어서 DTM을 2D 행렬로 변환
DTM_matrix = gensim.matutils.corpus2dense(DTM,num_terms=len(id2word)).T


# 5. DataFrame으로 변환
doc_names = ['문서' + str(i+1) for i in range(len(documents))]

# 칼럼별 생성
vocab = list(id2word.values())


dense_dtm_df = pd.DataFrame(DTM_matrix, index=doc_names, columns=vocab)

# DataFrame 출력
display(dense_dtm_df)

,기분이,나는,날씨는,매우,오늘,좋다.,마셨다.,정말,커피는,커피를,...,오늘은,주말에,했다.,독서는,좋아한다.,책을,볼,영화를,예정이다.,저녁에
문서1,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
문서2,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
문서3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
문서4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
문서5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0


# 2.TF-IDF 예제

## 2.1.sklearn

In [ ]:
# 필요한 라이브러리 불러오기
import numpy as np
from collections import defaultdict
import pandas as pd
from IPython.display import display

# 새로운 샘플 문서 리스트
documents = [
    "AI 기술은 날로 발전하고 있다. 많은 회사들이 AI 연구에 투자하고 있다.",
    "기후 변화는 심각한 문제로 떠오르고 있다. 우리는 모두 지구를 보호해야 한다.",
    "AI 기술과 기후 변화는 현대 사회에서 중요한 이슈이다.",
    "건강한 식습관은 우리의 삶의 질을 향상시킬 수 있다.",
    "AI와 기후 변화, 그리고 건강한 식습관 모두 중요한 문제들이다."
]



# DataFrame 출력
display(tfidf_df)

## 2.2.gemsim

In [ ]:
# 필요한 라이브러리 불러오기


# 새로운 샘플 문서 리스트
documents = [
    "AI 기술은 날로 발전하고 있다. 많은 회사들이 AI 연구에 투자하고 있다.",
    "기후 변화는 심각한 문제로 떠오르고 있다. 우리는 모두 지구를 보호해야 한다.",
    "AI 기술과 기후 변화는 현대 사회에서 중요한 이슈이다.",
    "건강한 식습관은 우리의 삶의 질을 향상시킬 수 있다.",
    "AI와 기후 변화, 그리고 건강한 식습관 모두 중요한 문제들이다."
]



# DataFrame 출력
display(tfidf_gensim_dense_df)


In [ ]:
# 필요한 라이브러리 불러오기


# 새로운 샘플 문서 리스트
documents = [
    "스마트폰은 현대인의 필수품이 되었다. 많은 사람들이 스마트폰을 사용하고 있다.",
    "기후 변화는 전 세계적인 문제로, 모든 국가가 이에 대응해야 한다.",
    "스마트폰 기술은 빠르게 발전하고 있으며, 다양한 어플리케이션이 개발되고 있다.",
    "건강한 식습관은 우리의 삶에 큰 영향을 미친다. 좋은 식습관은 건강을 지키는 열쇠이다.",
    "기후 변화와 건강한 식습관은 모두 우리의 삶에 중요한 영향을 미친다.",
    "스마트폰의 과도한 사용은 건강에 해로울 수 있다. 적절한 사용이 필요하다.",
    "기후 변화 문제는 더 이상 미래의 문제가 아니다. 현재 우리가 해결해야 할 문제이다.",
    "건강한 식습관과 규칙적인 운동은 우리의 신체적, 정신적 건강을 모두 증진시킬 수 있다.",
    "스마트폰은 우리의 삶을 편리하게 하지만, 동시에 중독 문제를 야기할 수 있다.",
    "기후 변화로 인해 많은 지역이 기상 이변을 겪고 있다."
]



# DataFrame 출력
display(tfidf_gensim_dense_df)

